In [1]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
data=np.load("bank_oversample.npz")
data.files

['arr_0', 'arr_1']

In [5]:
x=data["arr_0"]
y=data["arr_1"]

In [3]:
from sklearn.model_selection import train_test_split

In [7]:
from sklearn.ensemble import VotingClassifier,BaggingClassifier

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [9]:
log=LogisticRegression()
knn=KNeighborsClassifier(n_neighbors=3)
tree=DecisionTreeClassifier(max_depth=10)

In [12]:
df=pd.read_csv("bank_oversample_new.csv")

In [6]:
df.head()

,age,balance,day,duration,campaign,pdays,previous,job_blue-collar,job_entrepreneur,job_housemaid,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_other,poutcome_success,poutcome_unknown,result
0,30,1787,19,79,1,-1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
1,33,4789,11,220,1,339,4,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,35,1350,16,185,1,330,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,30,1476,3,199,4,-1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
4,59,0,5,226,1,-1,0,1,0,0,...,0,0,1,0,0,0,0,0,1,0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 43 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   age                  8000 non-null   int64
 1   balance              8000 non-null   int64
 2   day                  8000 non-null   int64
 3   duration             8000 non-null   int64
 4   campaign             8000 non-null   int64
 5   pdays                8000 non-null   int64
 6   previous             8000 non-null   int64
 7   job_blue-collar      8000 non-null   int64
 8   job_entrepreneur     8000 non-null   int64
 9   job_housemaid        8000 non-null   int64
 10  job_management       8000 non-null   int64
 11  job_retired          8000 non-null   int64
 12  job_self-employed    8000 non-null   int64
 13  job_services         8000 non-null   int64
 14  job_student          8000 non-null   int64
 15  job_technician       8000 non-null   int64
 16  job_unemployed       800

In [8]:
from sklearn.preprocessing import MinMaxScaler

In [9]:
col=["age","balance","day","pdays","duration"]

In [15]:
minmax=MinMaxScaler()

In [17]:
df[col]=minmax.fit_transform(df[col])

In [18]:
df.head()

,age,balance,day,duration,campaign,pdays,previous,job_blue-collar,job_entrepreneur,job_housemaid,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_other,poutcome_success,poutcome_unknown,result
0,0.161765,0.068455,0.600000,0.024826,1,0.000000,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
1,0.205882,0.108750,0.333333,0.071500,1,0.389908,4,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0.235294,0.062590,0.500000,0.059914,1,0.379587,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.161765,0.064281,0.066667,0.064548,4,0.000000,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
4,0.588235,0.044469,0.133333,0.073486,1,0.000000,0,1,0,0,...,0,0,1,0,0,0,0,0,1,0


In [19]:
x=df.drop("result",axis=1)
y=df["result"]

In [20]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.7,random_state=42)

In [21]:
from sklearn.ensemble import RandomForestClassifier

In [22]:
random=RandomForestClassifier()

In [23]:
random.fit(x_train,y_train)

RandomForestClassifier()

In [24]:
y_pred=random.predict(x_test)

In [25]:
from sklearn.metrics import accuracy_score

In [26]:
accuracy_score(y_test,y_pred)

0.9314285714285714

In [32]:
from sklearn.model_selection import GridSearchCV

In [36]:
var={"n_estimators":[10,20,30,40,50,60],
     "criterion":['gini',"entropy"],
    "max_depth":[1,2,3,4],
    "min_samples_split":[1,2,3,4],
    "min_samples_leaf":[1,2,3,4]
    }

In [37]:
param=GridSearchCV(estimator=random,param_grid=var,cv=4,n_jobs=1,verbose=1,scoring="accuracy")

In [38]:
param.fit(x_train,y_train)

Fitting 4 folds for each of 768 candidates, totalling 3072 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 3072 out of 3072 | elapsed:  4.7min finished


GridSearchCV(cv=4, estimator=RandomForestClassifier(), n_jobs=1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [1, 2, 3, 4],
                         'min_samples_leaf': [1, 2, 3, 4],
                         'min_samples_split': [1, 2, 3, 4],
                         'n_estimators': [10, 20, 30, 40, 50, 60]},
             scoring='accuracy', verbose=1)

In [39]:
pd.DataFrame(param.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_min_samples_leaf,param_min_samples_split,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
0,0.018713,0.011023,0.000000,0.000000,gini,1,1,1,10,"{'criterion': 'gini', 'max_depth': 1, 'min_sam...",NaN,NaN,NaN,NaN,NaN,NaN,768
1,0.023889,0.004194,0.000000,0.000000,gini,1,1,1,20,"{'criterion': 'gini', 'max_depth': 1, 'min_sam...",NaN,NaN,NaN,NaN,NaN,NaN,643
2,0.028960,0.005460,0.000000,0.000000,gini,1,1,1,30,"{'criterion': 'gini', 'max_depth': 1, 'min_sam...",NaN,NaN,NaN,NaN,NaN,NaN,642
3,0.033916,0.006557,0.000000,0.000000,gini,1,1,1,40,"{'criterion': 'gini', 'max_depth': 1, 'min_sam...",NaN,NaN,NaN,NaN,NaN,NaN,638
4,0.042661,0.005382,0.000000,0.000000,gini,1,1,1,50,"{'criterion': 'gini', 'max_depth': 1, 'min_sam...",NaN,NaN,NaN,NaN,NaN,NaN,635
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,0.061459,0.004732,0.007812,0.007812,entropy,4,4,4,20,"{'criterion': 'entropy', 'max_depth': 4, 'min_...",0.875000,0.881667,0.908333,0.870000,0.883750,0.014784,58
764,0.095818,0.000930,0.007825,0.007826,entropy,4,4,4,30,"{'criterion': 'entropy', 'max_depth': 4, 'min_...",0.883333,0.885000,0.896667,0.880000,0.886250,0.006278,35
765,0.115710,0.005606,0.011718,0.006765,entropy,4,4,4,40,"{'criterion': 'entropy', 'max_depth': 4, 'min_...",0.863333,0.885000,0.896667,0.876667,0.880417,0.012155,91
766,0.151097,0.008849,0.012039,0.006971,entropy,4,4,4,50,"{'criterion': 'entropy', 'max_depth': 4, 'min_...",0.863333,0.888333,0.891667,0.903333,0.886667,0.014577,34


In [40]:
param.best_estimator_

RandomForestClassifier(criterion='entropy', max_depth=4, min_samples_leaf=2,
                       min_samples_split=3, n_estimators=40)

In [41]:
rr=RandomForestClassifier(criterion='entropy', max_depth=4, min_samples_leaf=2,
                       min_samples_split=3, n_estimators=40)

In [42]:
rr.fit(x_train,y_train)

RandomForestClassifier(criterion='entropy', max_depth=4, min_samples_leaf=2,
                       min_samples_split=3, n_estimators=40)

In [43]:
y=rr.predict(x_test)

In [45]:
accuracy_score(y_test,y)

0.8816071428571428

In [48]:
from sklearn.metrics import confusion_matrix,precision_score,recall_score,classification_report

In [49]:
confusion_matrix(y_test,y)

array([[2428,  348],
       [ 315, 2509]], dtype=int64)

In [50]:
precision_score(y_test,y)

0.8781939096954847

In [51]:
recall_score(y_test,y)

0.8884560906515581

In [53]:
print(classification_report(y_test,y))

              precision    recall  f1-score   support

           0       0.89      0.87      0.88      2776
           1       0.88      0.89      0.88      2824

    accuracy                           0.88      5600
   macro avg       0.88      0.88      0.88      5600
weighted avg       0.88      0.88      0.88      5600

